In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/applied-ai-engineering-samples/blob/agents-api-notebooks/genai-on-vertex-ai/agents/vertex_ai_agent_api/notebooks/talk_to_codebase.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapplied-ai-engineering-samples%2Fagents-api-notebooks%2Fgenai-on-vertex-ai%2Fagents%2Fvertex_ai_agent_api%2Fnotebooks%2Ftalk_to_codebase.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/applied-ai-engineering-samples/agents-api-notebooks/genai-on-vertex-ai/agents/vertex_ai_agent_api/notebooks/talk_to_codebase.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/applied-ai-engineering-samples/blob/agents-api-notebooks/genai-on-vertex-ai/agents/vertex_ai_agent_api/notebooks/talk_to_codebase.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Shivam Somani](https://github.com/somanishivam) |
| | [Driti Singh](https://github.com/dritisingh) |
| | [Christos Aniftos](https://github.com/anifort) |
|Reviewer(s) | [Meltem Subasioglu](https://github.com/5y5tem) |
|Last updated | October 24, 2024 |

# Overview
In this notebook, we will show you how to chat with your codebase, webpage with code bugs and feature requests, and your documents such as code style guidelines using Agent API.

We will be setting up the following:
1. Codebase Ingestion into Gemini's Long Context
2. Web Browser Extension for finding reported code issues
3. Search Extension for Code Style document search





 ## Vertex AI Agent API

[Vertex AI Agent API](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/overview) is an API for creating and managing Generative AI systems called "agents" that can  reason, plan, and act to perform specific tasks.

 Vertex AI Agent API offers faster time to market than building agents from scratch while still being flexible and customizable. It handles orchestraction and state management, gives you the benefits of Google's expertise in building reliable AI systems, scales in a secure and responsible way, and seamlessly integrates with other Vertex AI and Google Cloud products.

## Using This Notebook

Colab is recommended for running this notebook, but it can run in any iPython environment where you can connect to Google Cloud, install pip packages, etc.

If you're running outside of Colab and encountering issues, the [Getting Started with Vertex AI Agent notebook](https://github.com/GoogleCloudPlatform/applied-ai-engineering-samples/blob/agents-api-notebooks/genai-on-vertex-ai/agents/vertex_ai_agent_api/notebooks/getting_started_vertex_agent_api.ipynb) has some troubleshooting tips.

This tutorial uses the following Google Cloud services and resources:

* [Vertex AI Agent API](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/overview)
* Cloud Storage API
* Vertex AI API
* Vertex AI Search API and Webpage Broswer Extensions

This notebook has been tested in the following environment:

* Python version = 3.10.12
* [google-cloud-aiplatform](https://pypi.org/project/google-cloud-aiplatform/) version = 1.5.5

**Note:** Vertex AI Extensions requires google-cloud-aiplatform version >= 1.47.0

## Useful Tips

1. This notebook uses Generative AI cababilities. Re-running a cell that uses Generative AI capabilities may produce similar but not identical results.
2. Because of #1, it is possible that an output produces errors. If that happens re-run the cell that produced the error. The re-run will likely be bug free.
3. The use of Generative AI capabilities is subject to service quotas. Running the notebook using "Run All" may exceed your queries per minute (QPM) limitations. Run the notebook manually and if you get a quota error pause for up to 1 minute before retrying that cell. The Vertex AI Agent API defaults to Gemini on the backend and is subject to the Gemini quotas, <a href="https://console.cloud.google.com/iam-admin/quotas?pageState=(%22allQuotasTable%22:(%22f%22:%22%255B%257B_22k_22_3A_22_22_2C_22t_22_3A10_2C_22v_22_3A_22_5C_22base_model_5C_22_22%257D_2C%257B_22k_22_3A_22_22_2C_22t_22_3A10_2C_22v_22_3A_22_5C_22gemini_5C_22_22%257D%255D%22%29%29&e=13802955&mods=logs_tg_staging" >view your Gemini quotas here</a>.


# Setup

## Enable APIs and Set Permissions
1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Service Usage API](https://console.cloud.google.com/apis/library/serviceusage.googleapis.com)
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).
1. [Enable the Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com).
1. [Enable the Discovery Engine API for your project](https://console.cloud.google.com/marketplace/product/google/discoveryengine.googleapis.com)
1. [Enable the Agent Builder API](https://console.cloud.google.com/gen-app-builder/start)

To run the complete Notebook, you will need to have the following [roles](https://cloud.google.com/iam/docs/granting-changing-revoking-access) for your project:

* **`roles/serviceusage.serviceUsageAdmin`** to enable APIs
* **`roles/iam.serviceAccountAdmin`** to modify service agent permissions
* **`roles/discoveryengine.admin`** to modify discoveryengine assets
* **`roles/aiplatform.admin`** to create and use AI Platform components
* **`roles/storage.objectAdmin`** to modify and delete GCS buckets



### Authenticating your notebook environment

TODO: Remove for Public Preview

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
    print("authenticated")

## Install the Google Cloud Vertex AI Python SDK

TODO: Update to public pip and retest notebook (in and out of colab) at public preview.

The code blocks below download and install the Vertex AI Agent API Python SDK.

In [ ]:
WHL_FILEPATH = "gs://vertex_sdk_private_releases/agents_v2/google_cloud_aiplatform-1.71.dev20241017+vertex.agents.v2-py2.py3-none-any.whl"
!gsutil cp {WHL_FILEPATH} .

# Install the private SDK
!pip install --quiet {WHL_FILEPATH.split("/")[-1]} "numpy<2.0.0" --force-reinstall

In [ ]:
#%pip install --quiet --upgrade google-auth
#%%pip install --quiet -U "pandas==2.2.2"
%pip install google-cloud-discoveryengine --upgrade

### Restart Runtime

You may need to restart your notebook runtime to use the Vertex AI SDK. You can do this by running the cell below, which restarts the current kernel.

You may see the restart reported as a crash, but it is working as-intended -- you are merely restarting the runtime.

The restart might take a minute or longer. After its restarted, continue to the next step.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Initialize the Google Cloud Vertex AI Python SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).


### Set Your Project ID

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID_HERE"  # @param {type:"string"}

### Set the Region

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

If you outside of Colab and are having problems authenticating, see the Authentication section in the [Getting Started with the Vertex AI Agent API](https://github.com/GoogleCloudPlatform/applied-ai-engineering-samples/blob/agents-api-notebooks/genai-on-vertex-ai/vertex_ai_agent_api/notebooks/getting_started_vertex_agent_api.ipynb) notebook.

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

### Import and Initialize the Vertex AI Python SDK

In [ ]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

# Setup Codebase context and Function tool for answering questions

## Step 1: Load Codebase as Context Caching for the LLM

This step ingests and parses codebase in long context.

Load codebase from a public github link.
By default in this demo we use the [Bank of Anthos](https://github.com/GoogleCloudPlatform/bank-of-anthos) but you can change the default git repo link to something else. As we do not use authentication with github in this notebook the githup link should be public.

The steos executed below are:
1. Create a bucket if it does not exists to upload the based code
1. Clone the git repo to the local storage where this notebook runs
1. Upload the code from local to the created bucket


In [ ]:
GIT_REPO = "https://github.com/GoogleCloudPlatform/bank-of-anthos" # @param {type:"string"}
CODE_FOLDER = GIT_REPO.split('/')[-1] #Default sub folder name in your gcs bucket. You can use this one.

In [ ]:
GUIDELINES_BUCKET = f"{PROJECT_ID}-coding-guidelines"

In [ ]:
# Create a GCS bucket if you don't have one
! set -x && gsutil mb -p $PROJECT_ID -l us-central1 gs://$GUIDELINES_BUCKET

In [ ]:
!git clone {GIT_REPO} && rm -rf ./{CODE_FOLDER}/.git

## Step 2: Parse code files

In this step we process the code files and we append them together to create a long context string. Non text base file types such as images will benerate a warning. You can ingnore those warning as our context should a consolidation of the text files found in the repository

In [ ]:
import glob, os

codebase = ""
id_counter = 1
for filename in glob.iglob(CODE_FOLDER + '**/**', recursive=True):
  if os.path.isfile(filename):
    try:
      with open(filename) as f:
        codebase += f"{filename}:\n{ f.read()}\nEOF\n\n"
        id_counter += 1
    except UnicodeDecodeError as e:
        print(f"Warning decoding {filename}: {e}")

print(f"\n #{id_counter} files processed. \nContext length of {len(codebase)}")


## Step 3: Create model with long context window and caching with a 30 minute TTL

Aleternative to vector store, and having availability of large token context, We will be creating a long context using [Context Caching](https://cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview). Cached context items, such as a large amount of text, an audio file, or a video file, can be used in prompt requests to the Gemini API to generate output

**RAG would be preferable in case of [supported files](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview#supported-doc-types). Since codebase have many different doc types RAG is not used here.

In [ ]:
import datetime
from vertexai.preview import caching
from vertexai.preview.generative_models import (
    #Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    #Tool,
)

In [ ]:
MODEL_NAME = 'gemini-1.5-flash-002' # @param {type:"string"}

In [ ]:
"""creates cache content object"""

SYSTEM_INSTRUCTION = (
    'You are an expert on a codebase, and your job is to answer '
    'the user\'s query based on the codebase you have access to'
)

cached_content = caching.CachedContent.create(
    model_name=MODEL_NAME,
    system_instruction=SYSTEM_INSTRUCTION,
    contents=[codebase],
    ttl=datetime.timedelta(minutes=30),
)

Create model reference with cached content

In [ ]:
model_with_cache = GenerativeModel.from_cached_content(cached_content=cached_content)

## Step 4: Defining Tool for Codebase answers using Function Declaration

In this section we are creating and defining a function that the agents can use to answer codebase queries.

In order for your agent to call a function, you need to provide some information about the function to call. You can use the FunctionDeclaration class to construct the function declaration you wish to use, and add this to your app when you create or update. You can read more on [function calling here.](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling)

First lets creating the function  for answering codebase queries using the model_with_cache that we created in the previous step.

In [ ]:
def talk_to_codebase(user_query: str):

    prompt= f"""
You are an AI assistant helping the users with their questions on a codebase.
Rules:
1. Give detailed information for the questions asked by users.
2. Do not make up the answer.
3. If the question is not related to the codebase, respond with a polite message that you can not help with the query and ask user if they have any question related to the codebase.

Question: {user_query}
    """
    response = model_with_cache.generate_content([prompt]).text
    return {"context": response}

Now we must create a function declaration. this declaration describes the function purpose and the interface (parameters it accepts). This way the agents know how to use this function. We will later pass this funciton declaration object to our agent

In [ ]:
talk_to_codebase_declaration = FunctionDeclaration(
    name="talk_to_codebase",
    description="Queries the Gemini model to answer questions about a codebase, providing detailed information and adhering to specific rules.",
    parameters={
        "type": "OBJECT",
        "properties": {
            "user_query": {
                "type": "STRING",
                "description": "The user's question about the codebase."
            }
        },
        "required": [
            "user_query"
        ]
    },
)

# Setup Document Search with coding Style Guides

In this part we are going to create a Vertex AI search engine which will be populated with coding style guides for 3 languages, Java, Python and Objective C.

For more information check our public documentation on [how to set up unstructured data store in vertex ai search](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#cloud-storage)

#### 1) Download PDFs and Ingest Into GCS Bucket

If you don't have gdown and gsutil, please install them. !pip install gdown, !pip install gsutil

In [ ]:
from google.cloud import storage
import gdown
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    generation_match_precondition = None
    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Create a GCS bucket if you don't have one


# Download the pdfs and ingest them to the GCS bucket
base_url = "https://raw.githubusercontent.com/GoogleCloudPlatform/applied-ai-engineering-samples/main/genai-on-vertex-ai/developer_productivity_with_genai/pdfs"
url_list = [
  f"{base_url}/Google%20Java%20Style%20Guide.pdf",
  f"{base_url}/Google%20Objective-C%20Style%20Guide.pdf",
  f"{base_url}/Google%20Python%20Style%20Guide.pdf"]

i=1
for url in url_list:
  filename_ = url.split('/')[-1].replace("%20","_")
  gdown.download(url, filename_)
  upload_blob(GUIDELINES_BUCKET,filename_,filename_)
  i+=1

#### 2) Create a Vertex AI Search Data Store
The Vertex AI Search extension needs a [Data Store and Vertex AI Search App](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest) to run. After creating the data store using the cell below, You can inspect it under: https://console.cloud.google.com/gen-app-builder/data-stores

In [ ]:
# Specify name, id, location for your datastore & Search engine id
DATA_STORE_NAME = "code-guidelines-datastore" # @param {type:"string"}
DATA_STORE_ID = "code-guidelines-datastore-id" # @param {type:"string"}
DATA_STORE_LOCATION = "global" #@param {type:"string"}

In [ ]:
from google.cloud import discoveryengine_v1 as discoveryengine
from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import AlreadyExists

def create_data_store(
    project_id: str,
    location: str,
    data_store_name: str,
    data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        solution_types=[discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH],
        content_config=discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED,
    )
    parent=client.collection_path(
            project_id, location, "default_collection"
        )

    request = discoveryengine.CreateDataStoreRequest(
        parent=parent,
        data_store=data_store,
        data_store_id=data_store_id,
    )



    try:
      operation = client.create_data_store(request=request)
      # Make the request
      # The try block is necessary to prevent execution from halting due to an error being thrown when the datastore takes a while to instantiate
      try:
          response = operation.result(timeout=90)
          print(response)
      except:
          print("long-running operation")
    except AlreadyExists as e:
      print(f"Datastore {data_store_name} under {parent} already exist.\nSkipping create opperation")


create_data_store(PROJECT_ID, DATA_STORE_LOCATION, DATA_STORE_NAME, DATA_STORE_ID) # datastore name can only contain lowercase letters, numbers, and hyphens

#### 3) Ingest PDF Files in the GCS bucket into the Vertex AI Search Data Store

In [ ]:
from google.api_core.client_options import ClientOptions
from typing import Optional

def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: Optional[str] = None) -> str:
    """Imports documents into a Vertex AI data store from GCS.
    This function imports documents into a specified data store within Vertex AI
    Agent Builder from a GCS bucket.
    """
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(client_options=client_options)
    # The full resource name of the search engine branch.
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",)

    request = discoveryengine.ImportDocumentsRequest(
        parent=parent,
        gcs_source=discoveryengine.GcsSource(
            input_uris=[gcs_uri], data_schema="content"
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,)
    # Make the request
    operation = client.import_documents(request=request)
    print(f"Waiting for operation to complete: {operation.operation.name}\nDepending on data size, this might take few minutes")
    response = operation.result()
    # Once the operation is complete, get information from operation metadata.
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)
    # Handle the response.
    print(response)
    print(metadata)
    return operation.operation.name

# Ingest pdfs in the GCS bucket to the data store
GCS_URI = f"gs://{GUIDELINES_BUCKET}/*.pdf"
import_documents(PROJECT_ID, "global", DATA_STORE_ID, GCS_URI)

#### 4) Create a Vertex Search App and Connect it to the Data Store
The following cell lets you create a Vertex AI Search App to ✨**connect**✨ to your newly created data store. Need to enable searchTier and searchAddOns for the Vertex AI Search Extension to work as shown in the cell below. Read more about [Advanced Features](https://cloud.google.com/generative-ai-app-builder/docs/about-advanced-features)


In [ ]:
SEARCH_APP_ID = "code-guidelines-datastore-engine" # @param {type:"string"}
SEARCH_APP_REGION = DATA_STORE_LOCATION

In [ ]:
def create_engine(
    project_id: str, location: str, data_store_id: str, search_engine_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.EngineServiceClient(client_options=client_options)

    # Initialize request argument(s)
    config = discoveryengine.Engine.SearchEngineConfig(
        search_tier=discoveryengine.SearchTier.SEARCH_TIER_ENTERPRISE, search_add_ons=[discoveryengine.SearchAddOn.SEARCH_ADD_ON_LLM]
    )

    engine = discoveryengine.Engine(
        display_name=search_engine_id,
        solution_type=discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        data_store_ids=[data_store_id],
        search_engine_config=config,
    )
    parent=discoveryengine.DataStoreServiceClient.collection_path(
              project_id, location, "default_collection"
    )
    try:
      request = discoveryengine.CreateEngineRequest(
          parent=parent,
          engine=engine,
          engine_id=engine.display_name,
      )
      # Make the request
      operation = client.create_engine(request=request)
      response = operation.result(timeout=90)
      print(response)

    except AlreadyExists as e:
      print(f"Search Engine {search_engine_id} under {parent} already exist.\nSkipping create opperation")


create_engine(PROJECT_ID, SEARCH_APP_REGION, DATA_STORE_ID, SEARCH_APP_ID)

After you create the search app, you can use search extension to connect to it to extract the coding best pracitces. Below cells show you how to get the information using search extension. You can read more about search extension [here](https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/vertex-ai-search).

In [ ]:
from google.cloud.aiplatform.private_preview.vertex_agents_v2.extensions import Extension

SEARCH_CONFIG  = f"projects/{PROJECT_ID}/locations/{SEARCH_APP_REGION}/collections/default_collection/engines/{SEARCH_APP_ID}/servingConfigs/default_search"
search_extension = Extension.from_hub(
    "vertex_ai_search",
    runtime_config={
        "vertex_ai_search_runtime_config": {
            "serving_config_name": SEARCH_CONFIG,
        }
    })

## Setting up Web Search for code issues
In this step we will be creating Web browser extension provided by Google. Webpage browser extension downloads information from the URL you specifcy in the prompt. We will use `Extension.from_hub(...)` to set up the extension.

We will specify in our prompt instructions that we want to only search the URL below which contains issues within the Bank of Anthos repository

In [ ]:
ISSUES_URL = "https://github.com/GoogleCloudPlatform/bank-of-anthos/issues" # @param {type:"string"}
webpage_browser_extension = Extension.from_hub(
"webpage_browser",
)

##Create an Agent with tools

In this section we will create ah Agent with vertex search, Webpage Browser extension, & function calling tool for talk-to-codebase. Agent will follow the instructions to call necessary tools.

In [ ]:
from google.cloud.aiplatform.private_preview.vertex_agents_v2 import agents,sessions

DISPLAY_NAME = "Developer Assist Agent"

INSTRUCTIONS = f"""
You are an expert developer assistant, who helps developer understand and answer question on codebase, search for relevant code issues and guide developer with coding style guide questions using only the
tools and extensions available to you.

Instructions:
1. Identify if question is asking about any issues or bugs or feature requests, if yes use the url: {ISSUES_URL} and send the question and url to the web browser extension.  Use the extension response to answer the question
2. Idenitfy if question is regarding a coding style, if yes Search using Search Extension. Use the extension response to answer the question
3. Identify if question is to understand or get information on codebase, then use talk_to_codebase extension. Use the extension response to answer the question
4. If question is unreleated to above, professionaly decline and tell you can only help with codebase, reported issues and coding style guides.

DO NOT MAKE UP ANSWERS ON YOUR OWN, ALWAYS USE TOOLS TO RETRIEVE INFORMATION BASED ON ABOVE INSTRUCTIONS AND CONSTRUCT ANSWER BASED ON INFORMATION"""

agent = agents.create(
    display_name=DISPLAY_NAME,
    instruction= INSTRUCTIONS,
    model="gemini-1.5-pro",
    tools=[
        webpage_browser_extension,
        talk_to_codebase_declaration,
        search_extension
    ],
)


A session represents a sequence of interactions between a user and an agent. Each conversation turn is composed of the following:

* A user action, in which the user provides the prompt.
* An agent action, in which the agent provides the response.
* Zero or more tool use actions, in which a tool is invoked.

In [ ]:
session = sessions.create(display_name="codebase-agent-session")

### Example #1. Search for issues and featrure requests using `webpage_browser_extension` tool

In [ ]:
session_response = session.create_run(agent=agent, content= "Describe the first 2 listed bugs of the bank of anthos repo?")
session_response

In [ ]:
print(str(session_response.steps[-1].content.parts[0].text))

In [ ]:
session_response = session.create_run(
    agent=agent,
    content= """
List all issues grouped by type. for example

Bugs:
1. detailed description of the bug...
2...

Feature Requests:
...
etc

""")

print(str(session_response.steps[-1].content.parts[0].text))

### Example #2. Query your codebase using `talk_to_codebase_declaration` tool

In [ ]:
session_response = session.create_run(agent=agent, content= "What are the python library requirements in codebase? list them in a requirements.txt format")
session_response

Agent here predicted a function call to talk_to_codebase with user_query parameter, lets call this function on client side and send back response to agent

In [ ]:
if session_response.state == "RUN_ACTION_REQUIRED":
  fc_content_parts = []
  for content_part in session_response.steps:
    fc = content_part.content.parts[0].to_dict().get("function_call", None)
    if fc is not None:
      print(f"calling function {fc['name']} with {fc['args']}")
      fc_content_parts.append(Part.from_function_response(
          name=fc["name"],
          response=locals()[fc["name"]](**fc["args"])))

  # continue the incomplete session run with the function calling reponses
  if len(fc_content_parts)>0:
    session_response = session.resume_run(
        run = session_response,
        content= fc_content_parts)

session_response

In [ ]:
print(str(session_response.steps[-1].content.parts[0].text))

Perfect! Now that we have an understanding on how function calling is triggered, lets pack the triggering of functions with the model call in a helper function. This function detects the need of function call and if needed, it will perform this action automatically and then return the final response. If there is no need for function calling it will simply return the initial model response. By combining the call and the processing and resuming the agent action we ensure that all our interactions with the agent reach a completed state.

In [ ]:
def get_content_with_function_call(agent, content):
  session_response = session.create_run(agent=agent, content=content)

  if session_response.state == "RUN_ACTION_REQUIRED":
    fc_content_parts = []
    for content_part in session_response.steps:
      fc = content_part.content.parts[0].to_dict().get("function_call", None)
      if fc is not None:
        fc_content_parts.append(Part.from_function_response(
            name=fc["name"],
            response=globals()[fc["name"]](**fc["args"])))

    # continue the incomplete session run with the function calling reponses
    if len(fc_content_parts)>0:
      session_response = session.resume_run(
          run = session_response,
          content= fc_content_parts)

  return session_response


Let's now ask few more questions

In [ ]:
session_response = get_content_with_function_call(agent=agent, content= "what is this codebase about?")
session_response.steps[-1].content.parts[0].text

In [ ]:
session_response = get_content_with_function_call(
    agent=agent,
    content= "looking into the basecode, which backend API returns the account balance")

print(str(session_response.steps[-1].content.parts[0].text))

In [ ]:
session_response = get_content_with_function_call(
    agent=agent,
    content= """
I want to add a warning in the login page to educate the user that the password should never be shared.
Which file or files do i need to change in the codebase?
can you suggest how the final code should be with the suggested changes?
""")

print(str(session_response.steps[-1].content.parts[0].text))

### Example #3. Query for code style guidance using search_extension (Vertex AI Search) Tool

In [ ]:
session_response = session.create_run(agent=agent, content= "Tell me more about the best java style according to the java coding style guide?")
session_response

In [ ]:
print(str(session_response.steps[-1].content.parts[0].text))

In [ ]:
session_response = session.create_run(agent=agent, content= "In java and in python, how do I name my methods? give me examples for a method called 'adding up numbers'")
print(str(session_response.steps[-1].content.parts[0].text))

#Cleanup

Clean up resources created in this notebook.


To delete Session, uncomment and run:


In [ ]:
# app.delete_session('<Session Name>')

To delete agent, uncomment and run. Using either the display name or the fully qualified resource name, you can delete a specific agent under the App.

In [ ]:
# app.delete_agent('Cymbal Real Estate Research Agent')

To delete your app, uncomment and run:


In [ ]:
# app.delete(app.app_name)

To delete both extensions you used in the notebook, uncomment and run:

In [ ]:
# webpage_browser_extension.delete()
# rag_extension.delete()

To delete the GCS bucket, uncomment and run:

In [ ]:
# Delete contents of the bucket and the bucket
#!gsutil -m rm -r gs://$GCS_BUCKET

## Delete Vertex Search App

To delete a datstore connected to a app you first need to delete the app and then the datastore.

In [ ]:
# def delete_engine_sample(
#     project_id: str,
#     location: str,
#     engine_id: str,
# ) -> str:
#     #  For more information, refer to:
#     # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
#     client_options = (
#         ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
#         if location != "global"
#         else None
#     )

#     # Create a client
#     client = discoveryengine.EngineServiceClient(client_options=client_options)

#     # The full resource name of the engine
#     # e.g. projects/{project}/locations/{location}/collections/default_collection/engines/{engine_id}
#     name = client.engine_path(
#         project=project_id,
#         location=location,
#         collection="default_collection",
#         engine=engine_id,
#     )

#     # Make the request
#     operation = client.delete_engine(name=name)

#     print(f"Operation: {operation.operation.name}")

#     return operation.operation.name

# delete_engine_sample(PROJECT_ID, SEARCH_APP_REGION, SEARCH_APP_ID)

## Delete the Data Store
It can take a couple of hours for the data store id to be released. If you are going to rerun the notebook, Either use a different id, or wait for a couple of hours, or completely skip creating the datastore again and use the alredy created one.

In [ ]:
# def delete_data_store(
#     project_id: str,
#     location: str,
#     data_store_id: str,
# ) -> str:
#     #  For more information, refer to:
#     # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
#     client_options = (
#         ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
#         if location != "global"
#         else None
#     )

#     # Create a client
#     client = discoveryengine.DataStoreServiceClient(client_options=client_options)

#     request = discoveryengine.DeleteDataStoreRequest(
#         # The full resource name of the data store
#         name=client.data_store_path(project_id, location, data_store_id)
#     )

#     # Make the request
#     operation = client.delete_data_store(request=request)

#     print(f"Operation: {operation.operation.name}")

#     return operation.operation.name


# delete_data_store(PROJECT_ID,DATA_STORE_LOCATION, DATA_STORE_ID)